In [268]:
import urllib.request
from datetime import datetime, timedelta
import uuid
import re
import pandas as pd

In [269]:
url = "file:///Users/ph290/Documents/HalloranSync/documents/for_mark/Services%20-%20PharmOutcomes180121.html"
d = datetime.today()
output_filename = '/Users/ph290/Documents/HalloranSync/documents/for_mark/output_'+d.strftime('%Y-%m-%d')+'.csv'
x = urllib.request.urlopen(url)
data = str((x.read()))

#remove like:
# Saved&nbsp;2021-01-11
data = re.sub('Saved&nbsp;2021-[0-1][0-9]-[0-3][0-9]', '', data)

text_file = open("/Users/ph290/Documents/HalloranSync/documents/for_mark/ignore_string_output.txt", "w")
text_file.write(data)
text_file.close()

In [270]:
out_dict = {}

d = datetime.today()

not_first_date = True

while not_first_date:
    date_given = d.strftime('%Y-%m-%d')
    if date_given in data:
        not_first_date = False
    else:
        d = d - timedelta(days=1)

date_exists = True

while d > datetime(2021, 1, 1, 0, 0, 0, 0):
#     print('loop0')

    date_given = d.strftime('%Y-%m-%d')
    print(date_given)

    if date_given in data:

        not_end = True
        i=1

        while not_end:
#             print('loop1')
            try:
                tmp = data.split(date_given)[i].split('NotClaimable')[0]
#                 tmp = data.split('NotClaimable')[0]
#                 tmp = data.split(date_given)[i].split('281483650')[0]
                try:
                    patient_name = tmp.split('info no-lock" title="')[1].split('">')[0]
                except IndexError:
                    patient_name = tmp.split('<span style="color:salmon;">&bull;&nbsp;')[1].split('<br>')[0]
                index = uuid.uuid4()
                out_dict[index]={}
                out_dict[index]['patient_name'] = patient_name
#                 if patient_name == 'James G Eadie':
#                     print(i)
#                     print(tmp)
#                 if patient_name == 'Brian E Sayers':
#                     print(i)
#                     print(tmp)
#                 if patient_name == 'JILL MURDIN':
#                     print(i)
#                     print(tmp)
                out_dict[index]['Date Given'] = date_given
                out_dict[index]['DOB'] = tmp.split('<br>&bull;&nbsp;')[1]
                out_dict[index]['Postcode'] = tmp.split('<br>&bull;&nbsp;')[2].split('</span>')[0]
                out_dict[index]['Vaccinator'] = tmp.split('provisionedit">')[5].split('</a>')[0]
                i += 1
#                 print(i)
            except IndexError:
                not_end = False

        d  =  d - timedelta(days=1)
    else:
        d  =  d - timedelta(days=1)

2021-01-15
2021-01-14
2021-01-13
2021-01-12
2021-01-11
2021-01-10
2021-01-09
2021-01-08
2021-01-07
2021-01-06
2021-01-05
2021-01-04
2021-01-03
2021-01-02
2021-01-01


In [271]:
#Read in pts_with_vax_appt
df_Ptswithvacappt = pd.read_csv('/Users/ph290/Documents/HalloranSync/documents/for_mark/Ptswithvacapptsuptoendof15thJan.csv') 
df_Ptswithvacappt['Date of Birth'] = pd.to_datetime(df_Ptswithvacappt['Date of Birth'])
df_Ptswithvacappt['Given Name'] = df_Ptswithvacappt['Given Name'].str.upper() 
df_Ptswithvacappt['Family Name'] = df_Ptswithvacappt['Family Name'].str.upper() 
df_Ptswithvacappt['Postcode'] = df_Ptswithvacappt['Postcode'].str.upper()
df_Ptswithvacappt = df_Ptswithvacappt.dropna()

df_out_dict = pd.DataFrame.from_dict(out_dict, orient='index')
df_out_dict['DOB'] = pd.to_datetime(df_out_dict['DOB'])
df_out_dict['Postcode'] = df_out_dict['Postcode'].str.upper() 
df_out_dict['patient_name'] = df_out_dict['patient_name'].str.upper()
df_out_dict.reset_index(drop=True, inplace=True)



In [281]:
#Identify individuals occuring multiple times in list
df = pd.DataFrame(columns=list(df_Ptswithvacappt))

for i in range(len(df_Ptswithvacappt.Postcode)):
    tmp = df_out_dict[(df_out_dict.Postcode == df_Ptswithvacappt.iloc[i].Postcode) & (df_out_dict['DOB'] == df_Ptswithvacappt.iloc[i]['Date of Birth']) & (df_out_dict["patient_name"].str.contains(df_Ptswithvacappt.iloc[i]['Family Name'])) & (df_out_dict["patient_name"].str.contains(df_Ptswithvacappt.iloc[i]['Given Name']))]
    if len(tmp) == 0:
        df = df.append(df_Ptswithvacappt.iloc[i])

        
# write DataFrame to an excel sheet 
df.to_csv('/Users/ph290/Documents/HalloranSync/documents/for_mark/vac_appoints_but_not_on_pharmaout.csv')
        

In [314]:
#As above but in other lists (i.e. lists swapped over)
df2 = pd.DataFrame(columns=list(df_out_dict))

for i in range(len(df_out_dict.Postcode)):
    tmp = df_Ptswithvacappt[(df_Ptswithvacappt.Postcode == df_out_dict.iloc[i].Postcode) & (df_Ptswithvacappt['Date of Birth'] == df_out_dict.iloc[i]['DOB']) & (df_Ptswithvacappt["Family Name"].str.contains(df_out_dict.iloc[i]['patient_name'].split(' ')[-1])) & 
                        (df_Ptswithvacappt["Given Name"].str.contains(df_out_dict.iloc[i]['patient_name'].split(' ')[0]))]
    if len(tmp) == 0:
#         print(i)
        df2 = df2.append(df_out_dict.iloc[i])
        
# write DataFrame to an excel sheet 
df2.to_csv('/Users/ph290/Documents/HalloranSync/documents/for_mark/on_pharmaout_but_not_vac_appoints.csv')
        

In [316]:
#Duplicate (or more) names
MyList =  []
for index in list(out_dict):
    item = out_dict[index]['patient_name']+' '+out_dict[index]['DOB']+' '+out_dict[index]['Postcode']
    MyList.append(item)
    
from collections import Counter

count_dict = dict(Counter(MyList))

for name, no in count_dict.items():  # for name, age in dictionary.iteritems():  (for Python 2.x)
    if no >= 2:
        print(name)

SyntaxError: unexpected EOF while parsing (<ipython-input-316-9bf6ca2b77f7>, line 13)

In [156]:
#output the data
with open(output_filename, 'w') as f:
    f.write("%s,%s,%s,%s,%s,\n"%('Date Given','Patient name','DOB','Postcode','Vaccinator'))
    for key in out_dict.keys():
#         f.write("%s,%s\n"%(key,[out_dict[key][key2] for key2 in (out_dict[key])]))
        f.write("%s,%s,%s,%s,%s,\n"%(out_dict[key]['Date Given'],out_dict[key]['patient_name'],out_dict[key]['DOB'],out_dict[key]['Postcode'],out_dict[key]['Vaccinator']))
